Census of Drug and Alcohol Treatment Services in Northern Ireland

In [191]:
from gssutils import *
scraper = Scraper('https://www.health-ni.gov.uk/publications/census-drug-and-alcohol-treatment-services-northern-ireland-2017')
scraper

## Census of drug and alcohol treatment services in Northern Ireland 2017

### Distributions

1. Drug and Alcohol Census 2017 ([application/pdf](https://www.health-ni.gov.uk/sites/default/files/publications/health/drug-alcohol-census-2017.pdf))
1. Data from Census of Drug and Alcohol Treatment Services ([MS Excel Spreadsheet](https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx))
1. Pre-release Access List Drug and Alcohol Census ([application/pdf](https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/pre-release-drug-alcohol-census.pdf))


In [192]:
tabs = {tab.name: tab for tab in scraper.distribution(
    title='Data from Census of Drug and Alcohol Treatment Services').as_databaker()}
tabs.keys()
tabs

{'Contents & Notes to tables': {<A4 'Contents'>, <A2 '1 March 2017'>, <A16 ''>, <B4 ''>, <A1 'Census of Drug and Alcohol Treatment Services in Northern Ireland:'>, <A19 3.0>, <B1 ''>, <B19 'Hospital Information System (HIS) figures relate to the number of emergency admissions to HSC hospitals for only drug-related conditions, only alcohol-related conditions or both drug and alcohol-related conditions, excluding admissions under specialties for mental health and learning disabilities.\xa0 These figures are as at 1st September 2016.'>, <B18 'All percentages are rounded to the nearest percentage point.'>, <A10 'Table 5'>, <A9 'Table 4'>, <B2 ''>, <B7 'Breakdown by Service Type'>, <B9 'Breakdown by Trust'>, <A14 ''>, <A7 'Table 2'>, <A13 ''>, <B6 'Breakdown by Age and Gender'>, <B15 ''>, <B8 'Breakdown by Residential Status'>, <A12 ''>, <B16 ''>, <A8 'Table 3'>, <B10 'Comparison Table'>, <A18 2.0>, <A11 ''>, <B13 ''>, <A17 1.0>, <B14 ''>, <B11 ''>, <A5 ''>, <A6 'Table 1'>, <B12 ''>, <B17 '

In [193]:
tables = []
for tab_name, script in [
    ('Table 1', 'Treatment Services by Age and Gender.ipynb'),
    ('Table 2', 'Treatment Services by Service Type(Age).ipynb'),
    ('Table 2', 'Treatment Services by Service Type(sex).ipynb'),
    ('Table 3', 'Treatment Services by Residential status(Age).ipynb'),
    ('Table 3', 'Treatment Services by Residential Status(sex).ipynb'),
    ('Table 4', 'Treatment Services by Trust(Age).ipynb'),
    ('Table 4', 'Treatment Services by Trust(sex).ipynb'),
    ('Table 5', 'Treatment Services by Comparison table.ipynb')
]:
    tab = tabs[tab_name]
    %run "$script"
    tables.append(new_table)

In [194]:
tidy = pd.concat(tables)
#tidy.to_csv('testCompare.csv', index = False)

In [195]:
tidy['Treatment Type'].fillna('Total', inplace = True)

In [196]:
tidy.count()

Period                          724
Sex                             724
Age                             724
Service Type                    724
Residential Status              724
Treatment Type                  724
Health and Social Care Trust    724
Measure Type                    724
Unit                            724
Value                           724
dtype: int64

In [197]:
tidy['Treatment Type'].unique()

array(['Alcohol Only', 'Drugs Only', 'Drugs & Alcohol', 'Total',
       'Under 18s', '18 and over'], dtype=object)

In [198]:
tidy.head()

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,1 March 2017,Persons,Under 18,all,all,Alcohol Only,all,Count,People,95
1,1 March 2017,Persons,Under 18,all,all,Drugs Only,all,Count,People,324
2,1 March 2017,Persons,Under 18,all,all,Drugs & Alcohol,all,Count,People,294
3,1 March 2017,Persons,Under 18,all,all,Total,all,Count,People,713
4,1 March 2017,Persons,18 and over,all,all,Alcohol Only,all,Count,People,2482


In [199]:
#tidy = tidy[tidy['Treatment Type'] == 'Alcohol Only']

In [200]:
tidy.head()

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,1 March 2017,Persons,Under 18,all,all,Alcohol Only,all,Count,People,95
1,1 March 2017,Persons,Under 18,all,all,Drugs Only,all,Count,People,324
2,1 March 2017,Persons,Under 18,all,all,Drugs & Alcohol,all,Count,People,294
3,1 March 2017,Persons,Under 18,all,all,Total,all,Count,People,713
4,1 March 2017,Persons,18 and over,all,all,Alcohol Only,all,Count,People,2482


In [201]:
tidy['Sex'].unique()
#tidy.to_csv('testCompare.csv', index = False)

array(['Persons', 'Male', 'Female', 'Male (%)', 'Female (%)',
       '% of Total', '% of all Males ', '% of all Females', 'Female  ',
       'All'], dtype=object)

In [202]:
tidy.drop(tidy[tidy.Sex.isin(['% of Total', '% of all Males ','% of all Females'])].index, inplace = True)

In [203]:
tidy['Sex'].unique()

array(['Persons', 'Male', 'Female', 'Male (%)', 'Female (%)', 'Female  ',
       'All'], dtype=object)

In [204]:
tidy['Period'].unique()

array(['1 March 2017', '1st March 2007', '1st March 2010',
       '1st March 2012', '1st September2014', '1st March 2017'],
      dtype=object)

In [205]:
from datetime import datetime

In [206]:
tidy.Period = pd.to_datetime(tidy.Period).dt.strftime('%Y-%m-%d')

In [207]:
tidy['Period'] = str('day/') + tidy['Period']

In [208]:
tidy['Age'].unique()
#tidy.to_csv('testCompare.csv', index = False)

array(['Under 18 ', '18 and over', 'All Ages', 'All', 'Under 18'],
      dtype=object)

In [209]:
tidy['Age'] = tidy['Age'].map(
    lambda x: {
        'Under 18 ' : 'under-18', 
        '18 and over' : '18-plus',
        'All years': 'all' ,
        'All': 'all'
        }.get(x, x))

In [210]:
#tidy['Treatment Type'] = 'alcohol'
#tidy.to_csv('testCompare.csv', index = False)

In [211]:
def user_perc(x,y):
    
    if ( (str(x) == 'Statutory (%)')) | ((str(x) == 'Non-statutory (%)')) | ((str(x) == 'Prison (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Service Type'], row['Measure Type']), axis = 1)



In [212]:
def user_perc(x,y):
    
    if ( (str(x) == 'Residential (%)')) | ((str(x) == 'Non-residential (%)')) | ((str(x) == 'Mixed (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Residential Status'], row['Measure Type']), axis = 1)



In [213]:
tidy['Health and Social Care Trust'].unique()

array(['all', 'All', 'Belfast', 'Northern', 'South Eastern', 'Southern',
       'Western', 'Northern (%)', 'South Eastern (%)', 'Southern (%)',
       'Western (%)', 'Emergency admissions (HIS) (%)',
       'Emergency admissions (HIS)  (%)',
       'Belfast + Northern + South Eastern + Southern + Western',
       'Prison', 'Emergency admissions (HIS)'], dtype=object)

In [214]:
def user_perc(x,y):
    
    if ( (str(x) == 'Belfast (%)')) | ((str(x) == 'Northern (%)')) | ((str(x) == 'South Eastern (%)')) | ((str(x) == 'Southern (%)')) | ((str(x) == 'Western (%)')) | ((str(x) == 'Emergency admissions (HIS) (%)')) : 
        
            return 'Percentage'
    else:
            return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Health and Social Care Trust'], row['Measure Type']), axis = 1)

In [215]:
def user_perc(x,y):
    
    if  ((str(x) == 'Emergency admissions (HIS)  (%)')) : 
        
            return 'Percentage'
    else:
            return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Health and Social Care Trust'], row['Measure Type']), axis = 1)

In [216]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].str.rstrip(' (%)')

In [217]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].str.lower()

In [218]:
tidy['Health and Social Care Trust'].unique()

array(['all', 'belfast', 'northern', 'south eastern', 'southern',
       'western', 'emergency admissions (his',
       'belfast + northern + south eastern + southern + western',
       'prison'], dtype=object)

In [219]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].map(
    lambda x: {
        'south eastern' : 'south-eastern', 
        'emergency admissions (his)' : 'emergency-admissions',
        'emergency admissions (his' : 'emergency-admissions'
        }.get(x, x))

In [220]:
tidy['Measure Type'] = tidy['Measure Type'].map(
    lambda x: {
        'Headcount' : 'Count', 
        'Percentage of Headcount' : 'Percentage',
        }.get(x, x))

In [221]:
tidy['Sex'].unique()

array(['Persons', 'Male', 'Female', 'Male (%)', 'Female (%)', 'Female  ',
       'All'], dtype=object)

In [222]:
def user_perc(x,y):
    
    if ( (str(x) == 'Male (%)')) | ((str(x) == 'Female (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Sex'], row['Measure Type']), axis = 1)



In [223]:
tidy['Sex'] = tidy['Sex'].map(
    lambda x: {
        'Female' : 'F', 
        'Male' : 'M',
        'Persons' : 'T',
        'Female  ' : 'F',
        'Male (%)' : 'M',
        'Female (%)': 'F'
        }.get(x, x))

In [224]:
tidy['Sex'] = tidy['Sex'].str.rstrip('(%)')

In [225]:
tidy['Service Type'].unique()

array(['all', 'Statutory', 'Non-statutory', 'Prison', 'Statutory (%)',
       'Non-statutory (%)', 'Prison (%)', 'All'], dtype=object)

In [226]:
tidy['Service Type'] = tidy['Service Type'].map(
    lambda x: {
        'Total' : 'all',
        'total' : 'all'
        }.get(x, x))

In [227]:
tidy['Service Type'] = tidy['Service Type'].str.rstrip(' (%)')

In [228]:
tidy['Service Type'] = tidy['Service Type'].str.lower()

In [229]:
tidy['Residential Status'] = tidy['Residential Status'].str.rstrip(' (%)')

In [230]:
tidy['Residential Status'] = tidy['Residential Status'].str.lower()

In [231]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].map(
    lambda x: {
        'total' : 'all' 
        }.get(x, x))

In [232]:
tidy['Residential Status'] = tidy['Residential Status'].map(
    lambda x: {
        'total' : 'all' 
        }.get(x, x))

In [233]:
tidy.head()

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,day/2017-03-01,T,under-18,all,all,Alcohol Only,all,Count,People,95
1,day/2017-03-01,T,under-18,all,all,Drugs Only,all,Count,People,324
2,day/2017-03-01,T,under-18,all,all,Drugs & Alcohol,all,Count,People,294
3,day/2017-03-01,T,under-18,all,all,Total,all,Count,People,713
4,day/2017-03-01,T,18-plus,all,all,Alcohol Only,all,Count,People,2482


In [236]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
tidy.to_csv(out / 'observations.csv', index = False)
\tidy.to_csv('testCompare.csv', index = False)
tidy['Treatment Type'].unique()

array(['Alcohol Only', 'Drugs Only', 'Drugs & Alcohol', 'Total',
       'Under 18s', '18 and over'], dtype=object)

In [235]:
tidy[tidy['Measure Type'] != 'Percentage'].to_csv(out / 'observations-no-percentages.csv', index=False)

There's a metadata tab in the spreadsheet with abstract and contact details. **Todo: extract these and also figure out what the license should really be.**

In [46]:
md_tab = tabs['Metadata']
md_heading = md_tab.filter('Metadata')
md_heading.assert_one()
properties = md_heading.fill(DOWN).is_not_whitespace()
values = properties.fill(RIGHT).is_not_whitespace()
headings = properties - values.fill(LEFT)
properties = properties - headings
cs = ConversionSegment(values, [
    HDim(properties, "Property", DIRECTLY, LEFT),
    HDim(headings, "Heading", CLOSEST, UP)
], includecellxy=True)
savepreviewhtml(cs)

In [47]:
from gssutils.metadata import THEME

for i, row in cs.topandas().iterrows():
    v = md_tab.get_at(row.__x, row.__y).value.strip()
    if row.Property == 'Abstract':
        scraper.dataset.description = v
    elif row.Property == 'National Statistics Theme':
        scraper.dataset.theme = THEME[pathify(v)]
    elif row.Property == 'Keyword':
        scraper.dataset.keyword = map(lambda x: x.strip(), v.split(' '))
    elif row.Property == 'Email Address':
        scraper.dataset.contactEmail = f'mailto:{v}'

In [48]:
scraper.dataset.family = 'health'
scraper.dataset.license = 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/'

with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())

In [49]:
tidy.to_csv('testCompare.csv', index = False)